In [59]:
# ============================================================
# 03_build_adp_boards.ipynb
# Build rookie + startup boards using draft-month time buckets
# Adds "long" CSV export without changing HTML output
# ============================================================
import os
import time
import re
from typing import Any, Optional, List

import requests
import pandas as pd
import numpy as np

# ----------------------------
# CONFIG
# ----------------------------
SEASON = 2024

# Choose time slice:
# - set MONTH = "ALL" to use full-season rollup
# - or set MONTH = "YYYY-MM" to use that draft month
MONTH = "ALL"

# If you'd rather do a rolling window, set ROLLING_MONTHS = 2 (or None)
ROLLING_MONTHS: Optional[int] = None

NUM_TEAMS = 12
STARTUP_ROUNDS = 35
ROOKIE_ROUNDS = 5

MIN_DRAFTS_STARTUP = 100
MIN_DRAFTS_ROOKIE  = 25

SCORING = "dynasty_2qb"   # dynasty_ppr / dynasty_half_ppr / dynasty_std etc
STARTUP_DRAFT_TYPE = "snake"
ROOKIE_DRAFT_TYPE  = "linear"

ROOT_DIR = "sleeper_dynasty_adp"
SNAP_DIR = os.path.join(ROOT_DIR, "data", "snapshots")
OUT_DIR  = os.path.join(ROOT_DIR, "data", "boards")
os.makedirs(OUT_DIR, exist_ok=True)

# ----------------------------
# HTTP (players)
# ----------------------------
BASE = "https://api.sleeper.app/v1"
session = requests.Session()
session.headers.update({"User-Agent": "Sleeper-Dynasty-ADP/1.0"})

def get_json(url: str, timeout: int = 30, retries: int = 4, backoff: float = 1.8) -> Any:
    last_err = None
    for i in range(retries):
        try:
            r = session.get(url, timeout=timeout)
            if r.status_code == 429:
                time.sleep(min(30, (backoff ** i) + 1))
                continue
            r.raise_for_status()
            return r.json()
        except Exception as e:
            last_err = e
            time.sleep(min(30, (backoff ** i) + 0.5))
    raise RuntimeError(f"GET failed: {url}\nLast error: {last_err}")

def url_players_nfl() -> str:
    return f"{BASE}/players/nfl"

# ----------------------------
# Load ADP time series
# ----------------------------
adp_path = os.path.join(SNAP_DIR, "adp_time_series", f"season={SEASON}", "adp_time_series.parquet")
if not os.path.exists(adp_path):
    raise FileNotFoundError(f"Missing {adp_path}. Run 01 or 02 first.")

adp_ts = pd.read_parquet(adp_path)
print("[OK] loaded adp_ts:", adp_ts.shape)

# Ensure expected cols exist
required_cols = [
    "season","start_month","dynasty_class","type","md_scoring_type","st_teams",
    "st_rounds","is_superflex","player_id","drafts","picks","adp","min_pick","max_pick"
]
missing = [c for c in required_cols if c not in adp_ts.columns]
if missing:
    raise KeyError(f"adp_ts missing required columns: {missing}")

adp_ts["start_month"] = adp_ts["start_month"].astype(str)
adp_ts["player_id"] = adp_ts["player_id"].astype(str)

# ----------------------------
# Select month slice
# ----------------------------
def select_time_slice(adp_ts: pd.DataFrame, month: str, rolling_months: Optional[int]) -> tuple[pd.DataFrame, str]:
    # If rolling, compute based on available monthly buckets (exclude ALL)
    if rolling_months is not None:
        months = sorted([m for m in adp_ts["start_month"].dropna().unique() if m != "ALL"])
        if not months:
            raise RuntimeError("No monthly buckets found (start_month).")
        keep = months[-int(rolling_months):]
        sl = adp_ts[adp_ts["start_month"].isin(keep)].copy()
        label = f"rolling_{rolling_months}_months_{keep[0]}_to_{keep[-1]}"
        return sl, label

    # else pick exactly MONTH
    sl = adp_ts[adp_ts["start_month"] == str(month)].copy()
    label = f"month_{month}"
    return sl, label

adp_slice, slice_label = select_time_slice(adp_ts, MONTH, ROLLING_MONTHS)
if adp_slice.empty:
    raise RuntimeError(f"No rows found for slice MONTH={MONTH} ROLLING_MONTHS={ROLLING_MONTHS}")

print("[OK] adp_slice:", adp_slice.shape, "slice_label:", slice_label)

# ----------------------------
# Consolidate across months (if rolling)
# ----------------------------
def consolidate_time_window(df: pd.DataFrame) -> pd.DataFrame:
    """
    Consolidate multiple month buckets into one ADP snapshot.
    Weighted mean ADP by picks: sum(adp * picks) / sum(picks)
    drafts/picks are summed.
    """
    keys = ["season","dynasty_class","type","md_scoring_type","st_teams","st_rounds","is_superflex","player_id"]

    tmp = df.copy()
    tmp["adp"] = pd.to_numeric(tmp["adp"], errors="coerce")
    tmp["picks"] = pd.to_numeric(tmp["picks"], errors="coerce").fillna(0)
    tmp["drafts"] = pd.to_numeric(tmp["drafts"], errors="coerce").fillna(0)

    # Weighted ADP numerator
    tmp["adp_x_picks"] = tmp["adp"] * tmp["picks"]

    out = (
        tmp.groupby(keys, dropna=False, as_index=False)
           .agg(
               drafts=("drafts","sum"),
               picks=("picks","sum"),
               adp_x_picks=("adp_x_picks","sum"),
               min_pick=("min_pick","min"),
               max_pick=("max_pick","max"),
           )
    )

    out["adp"] = out["adp_x_picks"] / out["picks"].replace(0, np.nan)
    out["adp"] = out["adp"].round(2)
    out = out.drop(columns=["adp_x_picks"])

    # If picks==0 for a group, adp will be NaN; drop them
    out = out[out["adp"].notna()].reset_index(drop=True)
    return out

if ROLLING_MONTHS is not None:
    adp_for_boards = consolidate_time_window(adp_slice)
else:
    adp_for_boards = adp_slice.copy()

print("[OK] adp_for_boards:", adp_for_boards.shape)

# ----------------------------
# Load players + clean
# ----------------------------
players = get_json(url_players_nfl())
players_raw = pd.DataFrame.from_dict(players, orient="index").reset_index().rename(columns={"index":"sleeper_player_id"})

if "player_id" in players_raw.columns:
    players_raw["player_id"] = players_raw["player_id"].where(players_raw["player_id"].notna(), players_raw["sleeper_player_id"])
else:
    players_raw["player_id"] = players_raw["sleeper_player_id"]

players_raw["player_id"] = players_raw["player_id"].astype(str)

want = ["player_id","full_name","position","team","years_exp","rookie_year","status"]
have = [c for c in want if c in players_raw.columns]
players_df = players_raw[have].copy()

for c in ["years_exp","rookie_year"]:
    if c in players_df.columns:
        players_df[c] = pd.to_numeric(players_df[c], errors="coerce")

# ----------------------------
# Helpers: pooling + boards
# ----------------------------
def collapse_adp_to_player_level(df: pd.DataFrame) -> pd.DataFrame:
    keys = [c for c in ["player_id","dynasty_class","md_scoring_type","is_superflex","st_teams","type"] if c in df.columns]
    out = (
        df.groupby(keys, as_index=False, dropna=False)
          .agg(
              adp=("adp", "mean"),
              picks=("picks", "sum"),
              drafts=("drafts", "sum"),
              min_pick=("min_pick", "min"),
              max_pick=("max_pick", "max"),
          )
    )
    out["adp"] = pd.to_numeric(out["adp"], errors="coerce")
    return out

def add_pos_rank_from_pool(pool: pd.DataFrame) -> pd.DataFrame:
    df = pool.copy()
    df["adp"] = pd.to_numeric(df["adp"], errors="coerce")
    df = df.sort_values(["position", "adp"], ascending=[True, True]).reset_index(drop=True)
    df["pos_rank"] = df.groupby("position").cumcount() + 1
    return df

def build_linear_board(pool: pd.DataFrame, num_teams: int = 12, num_rounds: int = 5) -> pd.DataFrame:
    need = num_teams * num_rounds
    ranked = pool.sort_values("adp").head(need).reset_index(drop=True)

    board = pd.DataFrame(
        index=[f"Round {r}" for r in range(1, num_rounds + 1)],
        columns=[f"Team {t}" for t in range(1, num_teams + 1)]
    )

    i = 0
    for r in range(1, num_rounds + 1):
        for t in range(1, num_teams + 1):
            if i >= len(ranked):
                break
            row = ranked.iloc[i]
            label = f"{r}.{t:02d}"  # 1.01 style for consistency
            board.loc[f"Round {r}", f"Team {t}"] = (
                f"{label} {row['full_name']}, {row['team']} {row['position']} ({row['pos_rank']})"
            )
            i += 1
    return board

def snake_picks(num_teams: int, num_rounds: int):
    picks = []
    for r in range(1, num_rounds + 1):
        for pick_in_round in range(1, num_teams + 1):
            team = pick_in_round if (r % 2 == 1) else (num_teams - pick_in_round + 1)
            picks.append({"round": r, "pick_in_round": pick_in_round, "team": team})
    return picks

def build_snake_board(pool: pd.DataFrame, num_teams: int = 12, num_rounds: int = 30) -> pd.DataFrame:
    need = num_teams * num_rounds
    ranked = pool.sort_values("adp").head(need).reset_index(drop=True)

    board = pd.DataFrame(
        index=[f"Round {r}" for r in range(1, num_rounds + 1)],
        columns=[f"Team {t}" for t in range(1, num_teams + 1)]
    )

    for i, pck in enumerate(snake_picks(num_teams, num_rounds)):
        if i >= len(ranked):
            break
        row = ranked.iloc[i]
        r = pck["round"]
        team = pck["team"]
        pir = pck["pick_in_round"]
        label = f"{r}.{pir:02d}"  # 1.01 etc

        board.loc[f"Round {r}", f"Team {team}"] = (
            f"{label} {row['full_name']}, {row['team']} {row['position']} ({row['pos_rank']})"
        )
    return board

# ----------------------------
# NEW: Convert board grid -> long CSV
# ----------------------------
def board_to_long_csv(board: pd.DataFrame) -> pd.DataFrame:
    """
    Convert a Round x Team board cell like:
      '1.01 Josh Allen, BUF QB (1)'
    into:
      Pick, Player, Team, Position, PositionalRank
    """
    pick_re = re.compile(
        r"^\s*(?P<pick>\d+\.\d+)\s+"
        r"(?P<player>.*?),\s+"
        r"(?P<team>[A-Z]{2,3})\s+"
        r"(?P<pos>[A-Z]{1,3})\s+"
        r"\((?P<pos_rank>\d+)\)\s*$"
    )

    rows = []
    for r_label in board.index:
        for c_label in board.columns:
            cell = board.loc[r_label, c_label]
            if pd.isna(cell) or str(cell).strip() == "":
                continue

            s = str(cell).strip()
            m = pick_re.match(s)
            if not m:
                rows.append({
                    "Pick": None, "Player": None, "Team": None, "Position": None, "PositionalRank": None,
                    "raw": s
                })
                continue

            d = m.groupdict()
            rows.append({
                "Pick": d["pick"],
                "Player": d["player"],
                "Team": d["team"],
                "Position": d["pos"],
                "PositionalRank": int(d["pos_rank"]),
            })

    out = pd.DataFrame(rows)

    # Sort numerically by pick
    def pick_sort_key(p):
        if pd.isna(p):
            return (9999, 9999)
        a, b = str(p).split(".")
        return (int(a), int(b))

    if not out.empty:
        out["_k"] = out["Pick"].apply(pick_sort_key)
        out = out.sort_values("_k").drop(columns=["_k"]).reset_index(drop=True)

    return out

# ----------------------------
# Pools (startup + rookie)
# ----------------------------
def get_startup_pool(adp_df: pd.DataFrame, players_df: pd.DataFrame) -> pd.DataFrame:
    df = adp_df[
        (adp_df["dynasty_class"] == "startup") &
        (adp_df["md_scoring_type"] == SCORING) &
        (adp_df["is_superflex"] == True) &
        (adp_df["st_teams"] == NUM_TEAMS) &
        (adp_df["type"] == STARTUP_DRAFT_TYPE)
    ].copy()

    if df.empty:
        return df

    p = collapse_adp_to_player_level(df)
    p["player_id"] = p["player_id"].astype(str)

    players_df2 = players_df.copy()
    players_df2["player_id"] = players_df2["player_id"].astype(str)

    p = p.merge(players_df2, on="player_id", how="left")

    p = p[p["position"].isin(["QB","RB","WR","TE"])].copy()
    p = p[p["team"].notna()].copy()
    p = p[p["drafts"] >= MIN_DRAFTS_STARTUP].copy()

    return p.sort_values("adp").reset_index(drop=True)

def get_rookie_pool(adp_df: pd.DataFrame, players_df: pd.DataFrame) -> pd.DataFrame:
    df = adp_df[
        (adp_df["dynasty_class"] == "rookie") &
        (adp_df["md_scoring_type"] == SCORING) &
        (adp_df["is_superflex"] == True) &
        (adp_df["st_teams"] == NUM_TEAMS) &
        (adp_df["type"] == ROOKIE_DRAFT_TYPE)
    ].copy()

    if df.empty:
        return df

    p = collapse_adp_to_player_level(df)
    p["player_id"] = p["player_id"].astype(str)

    players_df2 = players_df.copy()
    players_df2["player_id"] = players_df2["player_id"].astype(str)
    players_df2["_years_exp_num"] = pd.to_numeric(players_df2.get("years_exp"), errors="coerce")

    p = p.merge(players_df2[["player_id","full_name","team","position","_years_exp_num"]], on="player_id", how="left")
    p = p[p["position"].isin(["QB","RB","WR","TE"])].copy()

    # Eligible = 0–1 years experience
    p = p[p["_years_exp_num"].isin([0, 1])].copy()

    p = p[p["drafts"] >= MIN_DRAFTS_ROOKIE].copy()
    return p.sort_values("adp").reset_index(drop=True)

startup_pool = get_startup_pool(adp_for_boards, players_df)
rookie_pool  = get_rookie_pool(adp_for_boards, players_df)

startup_players = add_pos_rank_from_pool(startup_pool) if not startup_pool.empty else pd.DataFrame()
rookie_players  = add_pos_rank_from_pool(rookie_pool) if not rookie_pool.empty else pd.DataFrame()

startup_board = build_snake_board(startup_players, num_teams=NUM_TEAMS, num_rounds=STARTUP_ROUNDS) if not startup_players.empty else pd.DataFrame()
rookie_board  = build_linear_board(rookie_players, num_teams=NUM_TEAMS, num_rounds=ROOKIE_ROUNDS) if not rookie_players.empty else pd.DataFrame()

print("[OK] Startup pool:", startup_players.shape, "| Rookie pool:", rookie_players.shape)

# ----------------------------
# Save artifacts
# ----------------------------
out_startup = os.path.join(OUT_DIR, "startup", f"season={SEASON}", slice_label)
out_rookie  = os.path.join(OUT_DIR, "rookie",  f"season={SEASON}", slice_label)
os.makedirs(out_startup, exist_ok=True)
os.makedirs(out_rookie, exist_ok=True)

# Pools
if not startup_players.empty:
    startup_players.to_parquet(os.path.join(out_startup, "startup_player_pool.parquet"), index=False)
if not rookie_players.empty:
    rookie_players.to_parquet(os.path.join(out_rookie,  "rookie_player_pool.parquet"), index=False)

# HTML stays the same
if not startup_board.empty:
    startup_board.to_html(os.path.join(out_startup, "startup_snake_board.html"))
if not rookie_board.empty:
    rookie_board.to_html(os.path.join(out_rookie,  "rookie_linear_board.html"))

# CSV 1) grid output (same as before)
if not startup_board.empty:
    startup_board.to_csv(os.path.join(out_startup, "startup_snake_board.csv"))
if not rookie_board.empty:
    rookie_board.to_csv(os.path.join(out_rookie,  "rookie_linear_board.csv"))

# CSV 2) NEW long output (Pick, Player, Team, Position, PositionalRank)
if not startup_board.empty:
    startup_long = board_to_long_csv(startup_board)
    startup_long.to_csv(os.path.join(out_startup, "startup_snake_board_long.csv"), index=False)

if not rookie_board.empty:
    rookie_long = board_to_long_csv(rookie_board)
    rookie_long.to_csv(os.path.join(out_rookie, "rookie_linear_board_long.csv"), index=False)

print("[OK] wrote boards to:")
print("  ", out_startup)
print("  ", out_rookie)
print("[OK] wrote long CSVs:")
print("  ", os.path.join(out_startup, "startup_snake_board_long.csv"))
print("  ", os.path.join(out_rookie,  "rookie_linear_board_long.csv"))


[OK] loaded adp_ts: (417993, 14)
[OK] adp_slice: (116896, 14) slice_label: month_ALL
[OK] adp_for_boards: (116896, 14)
[OK] Startup pool: (395, 17) | Rookie pool: (12, 16)
[OK] wrote boards to:
   sleeper_dynasty_adp\data\boards\startup\season=2024\month_ALL
   sleeper_dynasty_adp\data\boards\rookie\season=2024\month_ALL
[OK] wrote long CSVs:
   sleeper_dynasty_adp\data\boards\startup\season=2024\month_ALL\startup_snake_board_long.csv
   sleeper_dynasty_adp\data\boards\rookie\season=2024\month_ALL\rookie_linear_board_long.csv
